<a href="https://colab.research.google.com/github/annaola/ZIwM/blob/master/ZIwM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowanie danych

In [ ]:
pip install pandas_ods_reader

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_ods_reader import read_ods
from sklearn.feature_selection import SelectKBest, chi2
from scipy.stats import mode
# pd.options.display.max_columns = None

## Pobieranie danych

In [ ]:
features = ["Koncentracja hemoglobiny", "Liczba erytrocytów", "Średnia objętość krwinki", "Średnie stężenie HB w krwince", "Wielkość erytrocytów", "Rodzaj erytrocytów", "Tkanka siateczkowata", "Szpik kostny", "Wielkość komórki", "Stosunek jądrowo-cytoplazmatyczny", "Rodzaj jądra", "Struktura chromatyny jądrowej", "Jąderko", "Pasożyty", "Ziarenka żelaza", "Poziom żelaza", "Poziom trwałych związków żelaza", "Poziom witaminy B", "Poziom kwasu foliowego", "Nieznana", "Reakcja odporonościowa", "Reakcja uro...", "Reakcja ruchliwości komórki", "Płeć", "Wiek", "Gorączka", "Krwawienie", "Skóra", "Węzły chłonne", "Szmery sercowe", "Wątroba, śledziona", "Klasa choroby"]
df = read_ods('./Anemia_dane/ANEMIA.ods', 1, columns=features)
df.astype(int)

## Podział danych na zbiór przykładów i ich klasy

In [ ]:
training_set = df.loc[:, features[0]:features[30]]
classes = df.loc[:, features[31]]

### Rozkład wartosci cech dla wszystkich przykładów

In [ ]:
training_set.describe()

### Rozkład wartosci cech z podziałem na choroby

In [ ]:
training_set_by_sikness = training_set.groupby(classes)
training_set_by_sikness.describe()

In [ ]:
training_set_by_sikness.agg(pd.Series.mode)

# Ranking cech

In [ ]:
chi2_stats, chi2_p = chi2(training_set, classes)
df_rank = pd.DataFrame(chi2_stats, training_set.columns).sort_values(0, ascending=False)
df_rank.round(2)

### Macierz korelacji

In [ ]:
data_corelated = df.corr()>=0.6
for c in data_corelated.columns:
    print("---- %s ---" % c)
    print(data_corelated[c].value_counts())

In [ ]:
df.corr()

# Tworzenie klasyfikatora i jego walidacja

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

def verifyClassifierWithParams(dataset, classes, layer_size, solver, momentum=0.0):
    # Można róznież poeksperymentować z różnymi funkcjami aktywacji
    classifier = MLPClassifier(hidden_layer_sizes=(layer_size, ), solver=solver, momentum=momentum, max_iter=10000)
    cross_validator = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=7890)
    scores = []

    for train_index, test_index in cross_validator.split(dataset, classes):
        X_train, X_test = dataset[train_index, : ], dataset[test_index, : ]
        y_train, y_test = classes.loc[train_index], classes.loc[test_index]
        classifier.fit(X_train, y_train)
        predict = classifier.predict(X_test)
        acc_score = accuracy_score(y_test, predict)
        scores.append(acc_score)

    mean_score = np.mean(scores)
    std_score = np.std(scores)
    print("Accuracy score: %.3f (%.3f)" % (mean_score, std_score))
    return mean_score

In [ ]:
layer_sizes = [15, 100, 300]
results_lbfgs = {"Number of features": [], "Layer size": [], "Result": []}
results_sgd = {"Number of features": [], "Layer size": [], "Result": []}

# klasyfikator osiąga najlepsze wyniki dla 7-8 cech, w dalszej przyszłości można zacząć sprawdzanie od 1 cechy
for k in range(5, 11):
#     Sprawdzanie accuracy dla optymalizatora wag 'lbfgs' - teoretycznie najlepszy dla zbiorów o wielkości zbliżonej do naszego, nie używa momentum
    for size in layer_sizes:
        train_set = SelectKBest(chi2, k=k).fit_transform(training_set, classes)
        print("------ Parametry: liczba cech = {}, layer_size = {}, solver='lbfgs'".format(k, size))
        results_lbfgs["Number of features"].append(k)
        results_lbfgs["Layer size"].append(size)
        results_lbfgs["Result"].append(verifyClassifierWithParams(preprocessing.scale(train_set), classes, size, 'lbfgs'))
        
# Sprawdzanie accuracy dla optymalizatora wag 'sgd' i momentum 0.8
    for size in layer_sizes:
            print("------ Parametry: liczba cech = {}, layer_size = {}, solver='sgd'".format(k, size))
            results_sgd["Number of features"].append(k)
            results_sgd["Layer size"].append(size)
            results_sgd["Result"].append(verifyClassifierWithParams(train_set, classes, size, 'sgd', 0.8))

In [ ]:
grid = sns.FacetGrid(pd.DataFrame(results_lbfgs), col="Layer size", col_wrap=4)
grid.map(plt.plot, "Number of features", "Result")

In [ ]:
grid = sns.FacetGrid(pd.DataFrame(results_sgd), col="Layer size", col_wrap=4)
grid.map(plt.plot, "Number of features", "Result")